In [1]:
from pyomo.environ import *
import pandas as pd
import yfinance as yfin

In [39]:
industries = {"MFST":"Technology", "AAPL":"Technology", "ASML":"Manufacturing", "TSLA":"Automobile", 'AMZN':"Technology",'JPM':'Financial Services','JNJ':'Pharmaceutical',
'MA':'Financial Services'}
dataframe = yfin.download(tickers=['MFST', 'AAPL', 'ASML', 'TSLA', 'AMZN', 'JPM','JNJ','MA'], period='1y')['Adj Close']

[*********************100%***********************]  8 of 8 completed


In [59]:
def optimisation_model(tickers, rolling_variance_dataframe, data_row, turnover_at_time_t, current_weights, industries, industry_bound
                       ):
    
    M = ConcreteModel()
    M.weights = Var(tickers, within=NonNegativeReals, bounds=(0,1), initialize=0)   
     
    M.Objective = Objective(
        expr=sum(data_row[s]*M.weights[s] for s in tickers),
        sense=-1
    )
        
    @M.Constraint()
    def portfolio_sum_to_one(M):
        return(sum(M.weights[s] for s in tickers)==1)
    
    @M.Constraint(tickers)
    def proper_weights(M, ticker):
        return(0, M.weights[ticker], 1)
    
    @M.Constraint()
    def turnover_constraint(M):
        return(-turnover_at_time_t, sum((M.weights[s] - current_weights[s]) for s in tickers), turnover_at_time_t)
    
    @M.Constraint()
    def industry_constraint(M):
        return(0, sum(M.weights[s] for s in industries.keys()), industry_bound)

    # 
    # @M.Constraint(S)
    # def active_security_bet(M, s):
    #     return(-L_security, M.w[s] - benchmark_weights[s], L_security)
    
    return M

In [60]:
# from more_itertools import windowed
import numpy as np
from tqdm.notebook import tqdm

opt = SolverFactory('glpk')

ROLLING_SIZE = 45
tickers = dataframe.columns.tolist()

rolling_variance_dataframe = dataframe.iloc[:ROLLING_SIZE, :].cov()
portfolio_weights = pd.DataFrame(columns=tickers, index=dataframe.index[ROLLING_SIZE:])

turnover_series = pd.Series(index=dataframe.index[ROLLING_SIZE:], dtype=float)
turnover_series.iloc[0] = 1
turnover_series.iloc[1:] = 0.1

current_weights = dict(zip(tickers, [0]*len(tickers)))

for t in tqdm(range(len(dataframe.iloc[ROLLING_SIZE:, :]))):
    
    state = True
    # print(f'{t}', end='')

    data_row = dataframe.iloc[t, :]
    turnover_at_time_t = turnover_series.iloc[t]
    
    industry_bound = 0.4

    while state:
        M = optimisation_model(tickers,
                               rolling_variance_dataframe,
                               data_row,
                               turnover_at_time_t,
                               current_weights,
                               industries, industry_bound
            #inputs to put into model
                            )
        opt_results = opt.solve(M, tee=False)

        if (opt_results.solver.status == SolverStatus.ok) and (opt_results.solver.termination_condition == TerminationCondition.optimal):
            pass
        else:
            print(f'WARNING: Unable to find solution')
            # Have to change some of the inputs we provide to the model here so that it converges, if we change nothing then it will be an infinite loop
            turnover_at_time_t += 0.05
            industry_bound+= 0.05
            continue

        state = False
    
    rolling_variance_dataframe = dataframe[t: ROLLING_SIZE + t].cov()
    df_weight_output_at_t = pd.DataFrame.from_dict(M.weights.extract_values(), orient='index', columns=[str(M.weights)])  

    # if np.allclose(df_weight_output_at_t.min(axis=1), 0.0):
    #     df_weight_output_at_t[df_weight_output_at_t<0] = 0
    
    portfolio_weights.iloc[t, :] = df_weight_output_at_t['weights'].values
    current_weights = portfolio_weights.iloc[t, :].to_dict()
    


  0%|          | 0/207 [00:00<?, ?it/s]

In [61]:
portfolio_weights

,AAPL,AMZN,ASML,JNJ,JPM,MA,MFST,TSLA
Date,,,,,,,,
2020-12-08,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-12-09,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-12-10,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-12-11,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-12-14,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
2021-09-28,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2021-09-29,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2021-09-30,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
turnover_series

Date
2020-12-08    1.0
2020-12-09    0.1
2020-12-10    0.1
2020-12-11    0.1
2020-12-14    0.1
             ... 
2021-09-28    0.1
2021-09-29    0.1
2021-09-30    0.1
2021-10-01    0.1
2021-10-04    0.1
Length: 207, dtype: float64